<h1><b>Statistique en Bioinformatique : </b> TME3 </h1>
<br>
L’objectif de ce TME est: 
<br>
<ul>
<li> objectif 1: comprendre les principaux etaphes de BLAST et PSIBLAST, </li>
<li> objectif 2: comprendre et comparer les algoritime de l'alignement progressive </li>
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME3.ipynb pour NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre sur moodle </li>
</ul>
</div>


Nom etudiant 1 : Antoine Grislain
<br>
Nom etudiant 2 : Simon Liétar
<br>

<h3>Section 1: BLAST</h3>

<b> Exercice 1 </b>: À partir de l'application BLAST, disponible via le site de <a href="http://blast.ncbi.nlm.nih.gov/">NCBI</a>, cliquez sur protein Blast. Faites un BLAST de la séquence fasta/seq1.fasta. Expliquez le résultat graphique, la couleur de chaque hit et expliquez le tableau de résultat qu'est-ce que c'est chaque colonne?
<br>


<font color="red"> Réponse : </font>
On retrouve sur le resultat graphique plusieurs hits qui font la taille de la sequence et de couleur rouge correspondant à 200 ou plus de score d'alignement.On observe aussi des séquences qui ont des hits roses au début et vert à la fin de la séquence. Les hits roses correspondent à un score d'alignement entre 80 et 200 et les verts à un score d'alignement entre 50 et 80.
Dans le tableau on trouve des informations pour chaque reads. Ces informations sont la description du read, son nom scientifique, son score max, son score total, le pourcentage de couverture de la sequence query, la E-value, le pourcentage d'identite, la longueur et le nombre de l'accession

<b>Exercice 2</b>: On s'intéresse à présent au gène FHIT chez l'homme (fragile histidine triad protein, identifiant P49789, télécharger du site  <a href="http://www.uniprot.org/"> Uniprot </a>). La protéine de ce gène est structurellement proche des galactose-1-phosphate uridylyltransferases. Faites un BLAST de ce gène contre la base SWISSPROT. Que constatez-vous?

<font color="red"> Réponse : </font>
On retrouve les FHIT de différents organismes avec de plutot bon scores mais on ne semble pas retrouver de galactose-1-phosphate uridylyltrans

<b>Exercice 3</b>: Implémenter l'étape 0 du algorithme BLAST (indexer la base de données), utilise w=5 et les séquences du fichier database.fasta.

In [1]:
import numpy as np

#variable initiation
database = 'fasta/database.fasta'
seqTestBlast = 'fasta/test.fasta'
input_blosum_f = 'BLOSUM62.txt'
aa = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'X']

#parameters
q = 21
gap = -5
word_length = 5


#For a simple test use:
databaseToy = 'fastaToy/databaseToy.fasta'
seqTestBlastToy = 'fastaToy/testToy.fasta'
wToy=3


In [2]:

from typing import Optional


def etape0(fileName: str, word_length: int):
    """
    Database indexation
    Input1 fileName: fasta file name
    Input2 w:  word size
    Output1 index: dictionary containing indexed words
    Output2 dicoSeq: dictionary containing database entire sequences
    """

    index_dict = dict[str, list[tuple[str, int]]]()
    sequences = dict[str, str]()

    current_seq_name: Optional[str] = None

    with open(fileName) as file:
        for raw_line in file:
            line = raw_line.strip()

            if line.startswith('>'):
                current_seq_name = line[1:].strip()
                sequences[current_seq_name] = ''
            elif line:
                assert current_seq_name is not None
                sequences[current_seq_name] += line.upper()

    for seq_name, seq_value in sequences.items():
        for offset in range(len(seq_value) - word_length + 1):
            word = seq_value[offset:(offset + word_length)]
            index_dict.setdefault(word, []).append((seq_name, offset))

    return index_dict, sequences

indexToy, dicoSeqToy = etape0(databaseToy, wToy)
print (indexToy)
print (dicoSeqToy)
#{'AGH': [('s1', 0)], 'CGG': [('s3', 0)], 'GGH': [('s2', 1)]}
#{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGH'}

{'AGH': [('s1', 0)], 'GHR': [('s1', 1), ('s2', 2)], 'CGG': [('s2', 0), ('s3', 0)], 'GGH': [('s2', 1), ('s3', 1)], 'GHA': [('s3', 2)]}
{'s1': 'AGHR', 's2': 'CGGHR', 's3': 'CGGHA'}


<b>Exercice 4</b>: Chercher la séquence query test.fasta dans la base indexé construite précédemment, entendez l'alignement pour avoir un alignement sans gap et de score > 12. Retournez les quatre meilleurs alignements.


In [3]:
#read Blosum
def readBlosum(nameFile: str):
	"""
	Read substitution matrix file
	Input1 fileName : blosum file name
	Output1 Blosum: a dictionary containing BLOSUM matrix, key is a pair of amino acids (alphabetic order) and value is the substitution value.
	REMARK : You could eliminate the last four columns and last four rows of Blosum file
	"""

	with open(nameFile, 'r') as in_f:
		lines = in_f.read().replace('  ', ' ').splitlines()
		data = [line[1:].strip() for line in lines[7:]]
		alphabet = lines[6].strip().split(' ')

	indices = [alphabet.index(a) for a in aa]
	matrix = np.loadtxt(data, dtype=int)
	matrix = matrix[:, indices][indices, :]

	assert matrix.shape[0] == matrix.shape[1] == q
	return matrix

blosum = readBlosum(input_blosum_f)
print(blosum)
# print (blosum["AA"])
#4

[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0  0]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3 -1]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2 -1]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3 -1]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -1]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -1]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2 -1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -1]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -1]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2]
 [ 1 -1  1

In [4]:
def extendAln(pos1: int, pos2: int, sequence1: str, sequence2: str, blosum_matrix: np.ndarray, scoremotif: float, w: int, threshold_score: float):
    """
    extend alignment left/right by keeping score above a threshold
    Input1 pos1 : position in the first sequence (Subject)
    Input2 pos2 : position in the second sequence (query)
    Input3 sequence1 : first sequence (Subject)
    Input4 sequence1 : second sequence (query)
    Input5 matriceBlosum: a dictionary containing BLOSUM matrix
    Input6 scoremotif: initial score
    Input7 w: word size
    Input8 ThrScore: threshold
    Output 1 finalScore: final score
    Output 2 algLength: alignment length
    Output 3 alnSeqs: Tuple containing aligned sequences
    """

    aa_inverse = { a: i for i, a in enumerate(aa) }
    current_score = scoremotif

    current_start1 = pos1
    current_start2 = pos2
    alignment_length = w

    while True:
        extend_left = (current_start1 > 0) and (current_start2 > 0)
        extend_right = (current_start1 + alignment_length < len(sequence1) - 1) and (current_start2 + alignment_length < len(sequence2) - 1)

        if (not extend_left) and (not extend_right):
            break

        if current_score < threshold_score:
            break

        if extend_left:
            current_start1 -= 1
            current_start2 -= 1

            current_score += blosum_matrix[
                aa_inverse[sequence1[current_start1]],
                aa_inverse[sequence2[current_start2]]
            ]

            alignment_length += 1

        if extend_right:
            current_score += blosum_matrix[
                aa_inverse[sequence1[current_start1 + alignment_length]],
                aa_inverse[sequence2[current_start2 + alignment_length]]
            ]

            alignment_length += 1

    return current_score, alignment_length, (
        sequence1[current_start1:(current_start1 + alignment_length)],
        sequence2[current_start2:(current_start2 + alignment_length)]
    )


res = extendAln(2, 2, dicoSeqToy['s2'], 'AGGHV', blosum, 6, 1, 5)
print (res)
#[20, 4, ('CGGH', 'AGGH')]

(20, 4, ('CGGH', 'AGGH'))


In [5]:
def getWordsQuery(word_length: int, query: str, indexB: dict[str, list[tuple[str, int]]]):
    """
    Loking for indexed words in the query
    Input1 w: word size
    Input2 query: sequence
    Input3 indexB: indexed database
    Output1 foundWords: Tuple containg found words, position in query, list of pairs (Subject id, Subject position)
    """

    return [
        (word, offset, indexB[word])
        for offset in range(len(query) - word_length + 1)
        if (word := query[offset:(offset + word_length)])
        in indexB
    ]

foundWords = getWordsQuery(wToy,  'AGGHV', indexToy)
print (foundWords)
#[('GGH', 1, [('s2', 1)])]


[('GGH', 1, [('s2', 1), ('s3', 1)])]


In [6]:
#Implement BLAST algorithm

def Blast(found_words: list[tuple[str, int, list[tuple[str, int]]]], indexedDB: dict[str, list[tuple[str, int]]], Query: str, dicoSeq: dict[str, str], threshold_score: float, word_length: int, blosum_matrix: np.ndarray):
    """
    input1 foundWords: Tuple containg found words, position in query, list of pairs (Subject id, Subject position
    input2 indexedDB: indexed database
    input3 query: query sequence
    input4 dicoSeq: dictionary containing database sequences
    input5 T: Threahold score
    input6 w: word size
    Input7 matriceBlosum: a dictionary containing BLOSUM matrix
    output1 alignedSeq: Tuple of aligned sequences
    """

    for word, query_offset, matches in found_words:
        for subject_name, subject_offset in matches:
            score, alignment_length, aligned_seq = extendAln(
                subject_offset,
                query_offset,
                dicoSeq[subject_name],
                Query,
                blosum_matrix,
                sum([blosum_matrix[aa.index(letter), aa.index(letter)] for letter in word]),
                word_length,
                threshold_score
            )

            if score > threshold_score:
                return aligned_seq

alignedSeq = Blast(foundWords, indexToy, "AGGHV", dicoSeqToy, wToy, 5, blosum)
print(alignedSeq)

('CGGHR', 'AGGHV')


<h3>Section 2: PSI-BLAST</h3>
On considère à nouveau le gène FHIT chez l'homme. 
<BR>
<b> Exercice 1 </b>:
Faites un PSI-BLAST de ce gène contre la base SWISS-PROT. Utilisez un seuil pour l’évalue de 1e-6.
Que constatez-vous lors de la 1ère itération concernant les protéines de la famille des galactose-1-phosphate uridylyltransferases?


<font color="red"> Réponse : </font>
On ne trouve aucunne protéine de cette famille dans les séquences qui ont matchées.


<b> Exercice 2 </b>:
Que constatez-vous lors de la 2eme itération? Lors de la 5eme itération?


<font color="red"> Réponse : </font>
On n'en trouve toujours aucunne à la deuxième itération. Et seulement une seule à la 5ème cela signifie que malgré des structures proches elles ont des séquences différentes.

<b> Exercice 3 </b>:
À partir de quelle itération peut-on considérer qu'il y a eu une convergence de l'algorithme PSI- BLAST? Pour quoi ?

<font color="red"> Réponse : </font>
Au bout de la 5ème itération on peut considérer qu'il y a convergence de l'algorithme car on ne trouve pas de nouveau résultats.

<h3>Section 3: Alignement Multiple</h3>


<b> Exercice 1 </b>:
Accédez à l'application <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a> et copiez les 7 séquences du fichier 7sequences.fasta. Expliquez, dans quel ordre les séquences ont été alignées.

<font color="red"> Réponse : </font>
SP|P02240|LGB2_LUPL
SP|P02185|MYG_PHYM
SP|P02208|GLB5_PETM
SP|P69905|HBA_HUMA
SP|P01958|HBA_HORS
SP|P68871|HBB_HUMA
SP|P02062|HBB_HORS
Les séquences ont été alignées en fonction de leur similarités en maximisant les régions similaires.

<b> Exercice 2 </b>:
Faite une fonction pour calculer le score d’un alignement multiple. Le score de chaque position est la somme des tous les scores obtenus par BLOSUM62, et le score final est la somme des scores de chaque position divisée par la taille du alignement.

In [7]:
#Read multi fasta
def extraireAllFastaMulti(fileName: str):
    """
    Read a FASTA file with several sequences
    input1 fileName: file name
    output1 IdSeq: list of sequences IDs
    output2 Seqs: sequences
    """

    sequences = dict[str, str]()
    current_seq_name: Optional[str] = None

    with open(fileName) as file:
        for raw_line in file:
            line = raw_line.strip()

            if line.startswith('>'):
                current_seq_name = line[1:].strip()
                sequences[current_seq_name] = ''
            elif line:
                assert current_seq_name is not None
                sequences[current_seq_name] += line.upper()

    return list(sequences.keys()), list(sequences.values())

print(extraireAllFastaMulti('fasta/algnMult.fasta'))

(['Sequence1', 'Sequence2', 'Sequence3', 'Sequence4', 'Sequence5'], ['GARFIELDTHELASTFATCAT', 'GARFIELDTHEFASTCAT', 'GARFIELDTHEVERYFASTCAT', 'THEFATCAT', 'GARFIELDTHEVASTCAT'])


In [8]:
import itertools

def SP_score(SeqsM: list[str], blosum: np.ndarray, gap: float):
    """
    Compute MSA score based on a substitution matrix and gap penality
    input1 SeqsM: list of aligned sequences
    input2 blosum: substitution matrix
    input3 gap: gap penality
    output1 score: MSA score
    """

    score = 0.0

    for offset in range(len(SeqsM[0])):
        score += sum([
            blosum[aa.index(i), aa.index(j)] if (i != '-') and (j != '-') else gap
            for i, j
            in itertools.combinations([seq[offset] for seq in SeqsM], 2)
        ])

    return score / len(SeqsM) # or len(SeqsM[0]) ?

SeqsM = ['ACG', 'A-G', 'ACD']
score = SP_score(SeqsM, blosum, gap)
print (score)
#5.0

5.0


<b> Exercice 3 </b>: Aligner les séquences du fichiers algnMult2.fasta avec  <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a>. Utiliser la fonction développé precedement  pour calculer le score du alignement. Quelle est la séquence la plus divergente? 

<font color="red"> Réponse : </font>
ptro|ENSPTRP00000012209 est la séquence la plus divergente.

<b> Exercice 4 </b>:
Comparer les trois outils  <a href="http://www.ebi.ac.uk/Tools/msa/clustalo/">Clustal omega</a>,  <a href="http://mafft.cbrc.jp/alignment/server/">MAFT</a>  et <a href="http://www.ebi.ac.uk/Tools/msa/muscle/">Muscle</a> en utilisant les séquences du fichiers algnMult.fasta. Utiliser la fonction développée précédemment pour calculer le score de chaque alignement. Puis répéter la comparaison avec les séquences du fichier algnMult2.fasta. Quel outil a eu la meilleure performance?

<font color="red"> Réponse : </font>
Les 3 algorithmes obtiennent des résultats très différents pour algnMult.fasta. mais pour algnMult2.fasta. les résultats sont plus proches. L'outil qui a été le plus performant est MUSCLE.